In [1]:
import undetected_chromedriver as uc
from time import sleep as time_sleep
from bs4 import BeautifulSoup as BS
import requests

In [2]:
url_list = '''

https://novelbin.com/b/shadow-slave
https://novelbin.com/b/strongest-necromancer-of-heavens-gate

'''.strip().split('\n')


url = url_list[0]

host = '/'.join(url.split('/', maxsplit=3)[:3])
name_title = url.split('/')[-1]
print(name_title)

chapters_link = f'{host}/ajax/chapter-option?novelId={name_title}&currentChapterId'
print(chapters_link)

shadow-slave
https://novelbin.com/ajax/chapter-option?novelId=shadow-slave&currentChapterId


In [3]:
list_links = list()

tt = requests.get(chapters_link)
links_value = BS(tt.text, 'html.parser')
tb_list = links_value.find_all('option')
elem : BS
for elem in tb_list:
    list_links.append(elem.get('value'))
print('Всего ссылок',len(list_links))
print('Последняя ссылка:')
print(list_links[-1])

Всего ссылок 827
Последняя ссылка:
https://novelbin.com/b/shadow-slave/chapter-827-daunting-task


In [13]:
driver = uc.Chrome()
driver.get(host)
time_sleep(1)
driver.get(url)
for _ in range(200):
    page = driver.page_source
    if not 'cloudflare.com' in page:
        break
    else:
        time_sleep(1)
else:
    driver.quit()
print('yes')



yes


In [14]:
book_eng = list()

page_start = -2

for link in list_links[page_start:]:
    driver.get(link)
    for _ in range(200):
        page = driver.page_source
        if not 'cloudflare.com' in page:
            break
        else:
            time_sleep(1)
    else:
        driver.quit()
    time_sleep(0.5)
    soup = BS(page, 'html.parser')
    novel_title = soup.find(attrs={'class':'novel-title'})
    content = soup.find(attrs={'id':'chr-content'})
    
    contents = content.find_all('p')
    
    content_list = list()
    el: BS
    for el in contents:
        content_list.append(el.get_text())
    text = '\n'.join(content_list)
    book_eng.append(text)
len(book_eng)

2

In [15]:
driver.quit()

In [18]:
book_eng_full = '\n'.join(book_eng)

In [16]:
# clean text:

book_eng_full = book_eng_full.replace('6054d257f56b520818c0fb96','')
book_eng_full = book_eng_full.replace('                  Ads by PubFuture','')
book_eng_full = book_eng_full.replace('The source of this content is','')
book_eng_full = book_eng_full.replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n')

In [11]:
# with open(
#     file=f'{name_title}.txt',
#     mode='w',
#     encoding='utf-8'
# ) as file:
#     file.write(book_eng_full)

In [19]:
from yandexfreetranslate import YandexFreeTranslate
from multiprocessing.dummy import Pool as ThreadPool
import json
from tqdm import tqdm

def _worker(que: tuple) -> str:
    queue, pbar = que
    if not queue:
        pbar.update(1)
        return ''
    yt = YandexFreeTranslate(api = "ios")
    text = yt.translate("en", "ru", queue)
    pbar.update(1)
    return text

def run_translate(lst: list[str])-> list[str]:
    with tqdm(total=len(lst) ) as pbar:
        l_b = list(zip(lst, ( pbar for _ in range(len(lst)))))
        with ThreadPool(30) as pool:
            workreturn = pool.map(_worker, l_b)
    return workreturn

lst = book_eng_full.split('\n')

text_rus = run_translate(lst)

text_rus = '\n'.join(text_rus)

with open(
    file=f'{name_title}_rus.txt',
    mode='w',
    encoding='utf-8'
) as file:
    file.write(text_rus)

100%|██████████| 96/96 [00:02<00:00, 38.33it/s]


In [12]:
# with open(
#     file=f'{name_title}_rus.txt',
#     mode='r',
#     encoding='utf-8'
# ) as file:
#     text = file.read()
# text = text.replace('вЂ”', '-').replace('вЂ"', '-').replace('вЂ“', '-')
# text = text.replace('вЂ¦', '…,')

# with open(
#         file=f'{name_title}_rus.txt',
#         mode='w',
#         encoding='utf-8'
#     ) as file:
#         file.write(text)